In [1]:
import sys
import os
import os.path


SPARK_HOME = """C:/spark-2.1.0-bin-hadoop2.7""" #CHANGE THIS PATH TO YOURS!

sys.path.append(os.path.join(SPARK_HOME, "python", "lib", "py4j-0.10.4-src.zip")) #BEWARE WITH py4j version!!
sys.path.append(os.path.join(SPARK_HOME, "python", "lib", "pyspark.zip"))
os.environ["SPARK_HOME"] = SPARK_HOME

from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("wind") \
    .config("spark.sql.caseSensitive", "false") \
    .getOrCreate()
    
sc = spark.sparkContext

In [4]:
# Import some libraries
import matplotlib.pyplot as plt 
# For plotting data
import numpy as np              
import os

from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import PCA
from pyspark.ml import Pipeline

os.getcwd()

'C:\\Users\\Clara\\Documents'

In [5]:
wind_sd = spark.read.csv(path="wind.csv", header=True, inferSchema=True)

In [5]:
wind_sd.printSchema()

root
 |-- energy: double (nullable = true)
 |-- steps: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- p54_162_1: double (nullable = true)
 |-- p54_162_2: double (nullable = true)
 |-- p54_162_3: double (nullable = true)
 |-- p54_162_4: double (nullable = true)
 |-- p54_162_5: double (nullable = true)
 |-- p54_162_6: double (nullable = true)
 |-- p54_162_7: double (nullable = true)
 |-- p54_162_8: double (nullable = true)
 |-- p54_162_9: double (nullable = true)
 |-- p54_162_10: double (nullable = true)
 |-- p54_162_11: double (nullable = true)
 |-- p54_162_12: double (nullable = true)
 |-- p54_162_13: double (nullable = true)
 |-- p54_162_14: double (nullable = true)
 |-- p54_162_15: double (nullable = true)
 |-- p54_162_16: double (nullable = true)
 |-- p54_162_17: double (nullable = true)
 |-- p54_162_18: double (nullable = true)
 |-- p54_162_19: double 

In [6]:
wind_sd.take(3)

[Row(energy=402.71, steps=0, year=2005, month=1, day=2, hour=18, p54_162_1=2534969.6319, p54_162_2=2526863.84971, p54_162_3=2518753.54925, p54_162_4=2510647.76706, p54_162_5=2502537.4666, p54_162_6=2531111.02655, p54_162_7=2522720.59315, p54_162_8=2514330.15975, p54_162_9=2505939.72635, p54_162_10=2497549.29295, p54_162_11=2528052.15557, p54_162_12=2519435.80851, p54_162_13=2510823.97972, p54_162_14=2502212.15093, p54_162_15=2493595.80387, p54_162_16=2525567.10529, p54_162_17=2516770.0273, p54_162_18=2507977.46758, p54_162_19=2499180.38959, p54_162_20=2490387.82987, p54_162_21=2522309.43029, p54_162_22=2513281.92036, p54_162_23=2504254.41043, p54_162_24=2495226.90051, p54_162_25=2486199.39058, p55_162_1=9.44171895697, p55_162_2=9.40377797844, p55_162_3=9.36651451737, p55_162_4=9.32857353883, p55_162_5=9.2906325603, p55_162_6=9.3475440281, p55_162_7=9.30689297967, p55_162_8=9.26624193124, p55_162_9=9.22491336533, p55_162_10=9.18426231689, p55_162_11=9.27369462345, p55_162_12=9.229655987

In [ ]:
# Write code here

In [ ]:
# Use as many cells as you need

In [21]:
#Creating the train, validation and test partitions

wind_train = wind_sd.filter(wind_sd['year'] <= 2006)
wind_val = wind_sd.filter(wind_sd['year'] > 2006).filter(wind_sd['year'] <= 2008)
wind_test = wind_sd.filter(wind_sd['year'] > 2008)

In [23]:
#Removing the 5 first columns - not needed

wind_train = wind_train.drop('steps', 'year', 'month', 'day', 'hour')
wind_val = wind_val.drop('steps', 'year', 'month', 'day', 'hour')
wind_test = wind_test.drop('steps', 'year', 'month', 'day', 'hour')

In [24]:
from pyspark.ml.feature import VectorAssembler

ignore = ['energy']
assembler = VectorAssembler(
    inputCols=[x for x in wind_train.columns if x not in ignore],
    outputCol='features')

wind_train = assembler.transform(wind_train).select(['energy', 'features'])

ignore = ['energy']
assembler = VectorAssembler(
    inputCols=[x for x in wind_val.columns if x not in ignore],
    outputCol='features')

wind_val = assembler.transform(wind_val).select(['energy', 'features'])

ignore = ['energy']
assembler = VectorAssembler(
    inputCols=[x for x in wind_test.columns if x not in ignore],
    outputCol='features')

wind_test = assembler.transform(wind_test).select(['energy', 'features'])

In [26]:
from pyspark.ml.feature import PCA
from pyspark.ml.feature import ChiSqSelector
from pyspark.ml import Pipeline
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator


In [33]:
# Pipeline 1 = pca 5 components => selector 3 features => decision tree 

results = []

for i in range(5, 100, 5):

    pca = PCA(k=i, inputCol="features")
    dt = DecisionTreeRegressor(featuresCol=pca.getOutputCol(), 
                               labelCol="energy")

    pipeline = Pipeline(stages=[pca, dt])

    model = pipeline.fit(wind_train)

    predictions = model.transform(wind_val)

    evaluator = RegressionEvaluator(
                labelCol="energy", 
                predictionCol="prediction", 
                metricName="mae")

    mae = evaluator.evaluate(predictions)
    
    results.append([i, mae])

results

min(results[1])#### FIND SOLUTION FOR THIS, MAYBE WITH ARGMIN?? TO GET THE INDEX IN WHICH THE MIN ELEMENT IS AND THEN GET THE KEY FROM SOME OTHER LIST

[[5, 469.96397500686055],
 [10, 470.60650390856256],
 [15, 474.7406058882686],
 [20, 474.84649722479975],
 [25, 474.4392560320692],
 [30, 474.4392560320692],
 [35, 447.9666423914338],
 [40, 447.84534776696665],
 [45, 446.74254591006724],
 [50, 448.43167475012575],
 [55, 448.43167475012575],
 [60, 449.695364817011],
 [65, 445.1972840388339],
 [70, 447.16127319306423],
 [75, 445.73909491083185],
 [80, 451.7444198943323],
 [85, 443.88448426773783],
 [90, 448.4568654434856],
 [95, 457.25423865382623]]

In [44]:
k_min = 85

wind_train_and_val = wind_sd.filter(wind_sd['year'] <= 2008)
wind_train_and_val = wind_train_and_val.drop('steps', 'year', 'month', 'day', 'hour')

ignore = ['energy']
assembler = VectorAssembler(
    inputCols=[x for x in wind_train_and_val.columns if x not in ignore],
    outputCol='features')

wind_train_and_val = assembler.transform(wind_train_and_val).select(['energy', 'features'])
wind_train_and_val = wind_train_and_val.withColumnRenamed('energy', 'label')

dt = DecisionTreeRegressor()
model=dt.fit(wind_train_and_val)

predictions_sd = model.transform(wind_test)

evaluator = RegressionEvaluator(
labelCol="energy", predictionCol="prediction", metricName="mae")
mae = evaluator.evaluate(predictions_sd)


print(str(mae) +  " MAE with all the attributes.")

321.794387566 MAE with all the attributes.


In [ ]:
spark.stop()